# Tarea 1
## IIC2440 - Procesamiento de Datos Masivos

Integrantes: 
- Rodrigo Nahum
- Fernando Quintana

Notebook 2: Obtención de personas que tweetean similar.

# Importar datos

Primero, importamos algunas librerias a usar.

In [42]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import pickle
import random

Igual que antes, importamos una funcion para ver el tamaño total de un objeto de Python. Obtenida de https://code.activestate.com/recipes/577504/

In [2]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

# Cargar datos

Primero, cargamos el csv, las hash signatures y los tweets unicos encontrados.

In [272]:
## Best parameters
# k = 5
# hash_funcs = 200
# b = 20
# r = 10

In [3]:
k = 5
hash_funcs = 200
b = 20
r = 10

filtered_data = pd.read_csv('filtered_tweets_2022_abril_junio.csv', index_col=0, quotechar='"')
reduced_data = filtered_data[filtered_data['text'].apply(len) >= k]

similar_items = set()
with open(f"similar_items_k={k}_hash_funcs={hash_funcs}_b={b}_r={r}.pkl", 'rb') as f:
    similar_items = pickle.load(f)

unique_tweets = []
with open(f"unique_tweets_k={k}.pkl", 'rb') as f:
    unique_tweets = pickle.load(f)

# Obtención de usuarios similares.

Armamos un diccionario, donde la llave sera un tweet, y el valor sera una lista con todos los usuarios que tweetearon eso. Recordemos que hay varios tweets que fueron tweeteados por mas de 1 persona.

In [4]:
tweets = {}
tweets_per_user = {}

for row in reduced_data.itertuples():

    try:
        if row.screen_name in tweets[row.text]:
            continue
        else:
            tweets[row.text].append(row.screen_name)
    except KeyError:
        tweets[row.text] = [row.screen_name]
    
    try:
        tweets_per_user[row.screen_name] += 1
    except KeyError:
        tweets_per_user[row.screen_name] = 1

Ahora, obtenemos los usuarios que tweetearon tweets similares. Para esto, mantenemos un diccionario que como llave tendra una tupla de 2 usuarios $(u_{1}, u_{2})$, y como valor tendra la cantidad de tweets similares entre esos 2 usuarios.

Para lograr esto, iteramos sobre todos los items similares $(i_{1}, i_{2})$, y para cada uno, obtenemos los usuarios que tweetearon el primer o segundo item. Luego, para cada par de usuarios $(u_{1}, u_{2})$ tal que $u_{1}$ tweeteo $i_{1}$ y $u_{2}$ tweeteo $i_{2}$, sumamos 1 al contador de $(u_{1}, u_{2})$.

In [7]:
similar_tweets_count = {}

for first_id, second_id in tqdm(similar_items):

    first_tweet = unique_tweets[first_id]
    second_tweet = unique_tweets[second_id]

    first_users = tweets[first_tweet]
    second_users = tweets[second_tweet]

    for first_user in first_users:
        for second_user in second_users:
            if first_user != second_user:
                try:
                    similar_tweets_count[(first_user, second_user)] += 1
                except KeyError:
                    similar_tweets_count[(first_user, second_user)] = 1

100%|██████████| 4716440/4716440 [00:22<00:00, 207169.19it/s]


In [8]:
total_size(similar_tweets_count)

1461006929

## Primer approach

Luego, obtenemos pares de personas que tweetean similar. Para esto, definimos dos umbrales. Primero, uno que corresponde a la cantidad mínima de tweets parecidos que ambas personas deben tener. Dicho de otra forma, la cantidad mínima de pares ($t_1$, $t_2$) tales que el primer usuario tweeteó $t_1$, el segundo tweeteó $t_2$, y $t_1$ y $t_2$ son similares.

El segundo umbral corresponde al porcentaje de tweets que deben ser parecidos. Esto es, el porcentaje (donde 100% corresponde a la suma de la cantidad de tweets de ambos usuarios) de pares de tweets parecidos entre los usuarios.

Si dos usuarios cumplen con ambos umbrales, entonces decimos que los usuarios tweetean de forma parecida.

In [9]:
absolute_threshold = 5
similar_threshold = 0.3

similar_users_final = set()

for key, item in tqdm(similar_tweets_count.items()):
    first_user_tweets = tweets_per_user[key[0]]
    second_user_tweets = tweets_per_user[key[1]]

    ## Si los usuarios cumplen con ambos umbrales
    if item >= similar_threshold * (first_user_tweets + second_user_tweets) and item >= absolute_threshold:
        ## Los agregamos de tal forma que el primer elemento sea
        ## siempre mayor al segundo, de nuevo para evitar duplicados.
        if key[0] > key[1]:
            similar_users_final.add(key)
        else:
            similar_users_final.add((key[1], key[0]))

100%|██████████| 13488359/13488359 [00:06<00:00, 2244883.47it/s]


In [10]:
len(similar_users_final)

249

De esta forma, obtenemos 249 pares de usuarios similares.

In [11]:
similar_users_final

{('AION01', '777SHEM'),
 ('ArkonteR', '777SHEM'),
 ('CarlosKeppeler', 'Ana63490092'),
 ('CarmenGordonOfi', 'ArrendadoresC'),
 ('Cony_V', 'AlikaSukni'),
 ('Cotalvez', '777SHEM'),
 ('Cristobalitoo', 'Cony_V'),
 ('Daniiconsu', '777SHEM'),
 ('DaphneLack', '777SHEM'),
 ('Defenixacamale1', '777SHEM'),
 ('FFQuakeProject', '777SHEM'),
 ('FFQuakeProject', 'AION01'),
 ('FFQuakeProject', 'Cotalvez'),
 ('FFQuakeProject', 'Defenixacamale1'),
 ('FafuC', '777SHEM'),
 ('Fernand95382651', '777SHEM'),
 ('Fernand95382651', 'FFQuakeProject'),
 ('FigurasYeso', 'ArrendadoresC'),
 ('FranCarbone86', '777SHEM'),
 ('FranCarbone86', 'FFQuakeProject'),
 ('FrenteFantasma', '777SHEM'),
 ('FrenteFantasma', 'FFQuakeProject'),
 ('GEOWASHJOR', '777SHEM'),
 ('GEOWASHJOR', 'FFQuakeProject'),
 ('HorstWenzel7', 'FromAnAU'),
 ('HorstWenzel7', 'GaoliferOliver'),
 ('IreneZaiferluck', '777SHEM'),
 ('JaimeMedinaAlva', 'AlikaSukni'),
 ('JaimeMedinaAlva', 'Cony_V'),
 ('LAGRINGAMERCED1', '15_rosana'),
 ('LAGRINGAMERCED1', 'CelinaV

A modo de ejemplo, mostramos los tweets de un par de usuarios similares según lo de arriba.

In [28]:
list(reduced_data[reduced_data['screen_name'] == 'CarmenGordonOfi'].text)

['RT @LuisaSepulvedaA: @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @CarmenGordonOfi: @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @Segpres @Arrendado…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @Giorgi

In [29]:
list(reduced_data[reduced_data['screen_name'] == 'ArrendadoresC'].text)

['RT @LuisaSepulvedaA: @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @LuisaSepulvedaA: @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @namegeorge: @skc000 @LuisaSepulvedaA @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_c…',
 'RT @skc000: @LuisaSepulvedaA @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJa…',
 'RT @LuisaSepulvedaA: @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…',
 'RT @Rossana90895427: @Emaa20188276 @Raul_Soto1 @Marcemomo @hector_ulloaa @gabrielboric @i_krmns @ArrendadoresC necesitamos, rapidez y fluid…',
 'RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @Giorgi

## Segundo approach.
Un problema con lo de arriba son las personas que tienen muchos tweets casi iguales (como 777SHEM). Esto hace que si otra persona tiene 1 tweet similar a alguno de 777SHEM, entonces ese tweet es similar a todos los de 777SHEM, lo que hace que se cumplan ambos umbrales, y sean reportados como pares similares.

Para solucionar esto, proponemos el siguiente approach. Ahora, mantenemos un diccionario cuyas llaves son un par de usuarios ($u_1$, $u_2$), y su valor corresponde a los tweets de $u_1$ que son parecidos a al menos un tweet de $u_2$. Notemos que ahora el orden si importa, o sea el valor de la llave ($u_1$, $u_2$) es distinto al de ($u_2$, $u_1$). Además, como guardamos una lista en vez de solamente un contador, esto también usa mucha más memoria. Sin embargo, ahora podemos saber la cantidad de tweets de $u_1$ que son similares a al menos un tweet de $u_2$ y vice-versa (con el approach anterior, no podíamos saber esto). Entonces, podemos definir mejor las condiciones de los umbrales. Podemos definir que dos usuarios tweetean similar cuando al menos un s% de los tweets de $u_1$ son similares a algún tweet de $u_2$ y vice-versa, lo que soluciona el problema anterior.

In [32]:
similar_tweets_second_approach = {}

for first_id, second_id in tqdm(similar_items):

    first_tweet = unique_tweets[first_id]
    second_tweet = unique_tweets[second_id]

    first_users = tweets[first_tweet]
    second_users = tweets[second_tweet]

    for first_user in first_users:
        for second_user in second_users:
            if first_user != second_user:
                try:
                    similar_tweets_second_approach[(first_user, second_user)].append(first_id)
                    similar_tweets_second_approach[(second_user, first_user)].append(second_id)
                except KeyError:
                    similar_tweets_second_approach[(first_user, second_user)] = [first_id]
                    similar_tweets_second_approach[(second_user, first_user)] = [second_id]
    


100%|██████████| 4716440/4716440 [00:54<00:00, 85933.31it/s] 


In [33]:
total_size(similar_tweets_second_approach)

4748835645

Usar esta estrategia requiere 4.7GB de memoria.

In [35]:
absolute_threshold = 5
similar_threshold = 0.3

similar_users_final_test = set()

for key, item in tqdm(similar_tweets_second_approach.items()):
    if key[0] > key[1]:
        continue

    second_user_similar = similar_tweets_second_approach[(key[1], key[0])]

    ## Cantidad de tweets del primer usuario
    first_user_tweets = tweets_per_user[key[0]]
    ## Cantidad de tweets del segundo usuario
    second_user_tweets = tweets_per_user[key[1]]

    ## Cantidad de tweets del primer usuario parecidos a
    ## algun tweet del segundo usuario
    first_match_count = len(set(item))
    ## Cantidad de tweets del segundo usuario parecidos a
    ## algun tweet del primer usuario
    second_match_count = len(set(second_user_similar))
    
    if first_match_count > similar_threshold * first_user_tweets and second_match_count > similar_threshold * second_user_tweets and \
        first_match_count > absolute_threshold and second_match_count > absolute_threshold:
        similar_users_final_test.add(key)


100%|██████████| 24733290/24733290 [00:18<00:00, 1327160.37it/s]


In [36]:
len(similar_users_final_test)

27

Esta vez, obtenemos solo 27 usuarios.

In [37]:
similar_users_final_test

{('777SHEM', 'FFQuakeProject'),
 ('777SHEM', 'jeannettekaz'),
 ('AlikaSukni', 'Cony_V'),
 ('AlikaSukni', 'Maritza11015251'),
 ('AlikaSukni', 'erikavarlop'),
 ('AlikaSukni', 'fiscalesdechile'),
 ('AlikaSukni', 'sanchezdetalca'),
 ('Cony_V', 'Maritza11015251'),
 ('Cony_V', 'erikavarlop'),
 ('Cony_V', 'fiscalesdechile'),
 ('Cony_V', 'sanchezdetalca'),
 ('FFQuakeProject', 'jeannettekaz'),
 ('FreeMitko', 'MITERRED'),
 ('Ilustradocl', 'andystefano'),
 ('Mariana_SanBK', 'Marioacunabuin'),
 ('Mariana_SanBK', 'duende_rebel'),
 ('Marioacunabuin', 'duende_rebel'),
 ('Maritza11015251', 'erikavarlop'),
 ('Maritza11015251', 'fiscalesdechile'),
 ('Maritza11015251', 'sanchezdetalca'),
 ('OmNoticiasAr', 'OmRadioAr'),
 ('PTV_Noticias', 'PanamericanaTV'),
 ('erikavarlop', 'fiscalesdechile'),
 ('erikavarlop', 'libra_74'),
 ('erikavarlop', 'sanchezdetalca'),
 ('fiscalesdechile', 'libra_74'),
 ('fiscalesdechile', 'sanchezdetalca')}

A modo de ejemplo, mostramos 2 usuarios parecidos.

In [40]:
list(reduced_data[reduced_data['screen_name'] == 'Ilustradocl'].text)

['Desde “pervertido” a “escombro”: la fuerte pelea entre Fulvio Rossi y Jorge Baradit en\xa0Twitter https://t.co/nBfPZMmRMq',
 'La ironía de Jorge Baradit tras llamativo discurso en la\xa0Convención https://t.co/mbH4l3dtyN',
 'Madre de Boric: “Gabriel se inmoló por la Convención\xa0Constitucional” https://t.co/mTZf3nUeEn',
 'Estudio indica que 58% de las personas ha recibido información falsa sobre la Convención\xa0Constitucional https://t.co/xsGp285rmc',
 'Javier Olivares indignado con la Convención Constitucional y dice que quieren eliminar a\xa0“Carabineros” https://t.co/vUa1ihvMO2',
 'Teresa Marinovic confirma que donará todo el dinero que ha ganado en la Convención\xa0Constitucional https://t.co/YnjA7d5KIb',
 'Ministra Jara y presidenta de la Convención Constitucional encabezaron marcha del Día del\xa0Trabajador https://t.co/76cbp2yR4W',
 'El nuevo pataleo de Teresa Marinovic: rechazó respaldo de Izkia Siches a la Convención\xa0Constitucional\u200b https://t.co/NF0BQ3mWcb',
 'Guar

In [41]:
list(reduced_data[reduced_data['screen_name'] == 'andystefano'].text)

['La ironía de Jorge Baradit tras llamativo discurso en la\xa0Convención https://t.co/U6kUfx2hgF',
 'Desde “pervertido” a “escombro”: la fuerte pelea entre Fulvio Rossi y Jorge Baradit en\xa0Twitter https://t.co/ndDdTsb1Fj',
 'Madre de Boric: “Gabriel se inmoló por la Convención\xa0Constitucional” https://t.co/7jTjErNU1M',
 'Estudio indica que 58% de las personas ha recibido información falsa sobre la Convención\xa0Constitucional https://t.co/9K5sQ2jS02',
 'Javier Olivares indignado con la Convención Constitucional y dice que quieren eliminar a\xa0“Carabineros” https://t.co/TA0febn6kw',
 'Teresa Marinovic confirma que donará todo el dinero que ha ganado en la Convención\xa0Constitucional https://t.co/2uxQ3WchYO',
 'El nuevo pataleo de Teresa Marinovic: rechazó respaldo de Izkia Siches a la Convención\xa0Constitucional\u200b https://t.co/6ep9cSxOfw',
 'Rayan monumento de Huanchaca en contra de la Convención Constitucional\xa0|FOTO https://t.co/CQAAGyNkli',
 'Guarello dice que Baradit co

## Resultados

A continuación, dejamos un código para visualizar los tweets de usuarios parecidos.

In [47]:
similar_users = ('CarmenGordonOfi', 'ArrendadoresC')

first_user_tweets = list(reduced_data[reduced_data['screen_name'] == similar_users[0]].text)
second_user_tweets = list(reduced_data[reduced_data['screen_name'] == similar_users[1]].text)

print("First user tweets:")
for item in first_user_tweets:
    print(item)

print()

print('Second user tweets:')
for item in second_user_tweets:
    print(item)


First user tweets:
RT @LuisaSepulvedaA: @CarmenGordonOfi @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…
RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…
RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…
RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…
RT @CarmenGordonOfi: @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @Segpres @Arrendado…
RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackson @S…
RT @CarmenGordonOfi: @LuisaSepulvedaA @chilevision @ppddiputados @PCdeChile @elfrente_amplio @ChilePDGcl @Presidencia_cl @GiorgioJackso